source: https://jorisvandenbossche.github.io/blog/2018/05/28/scikit-learn-columntransformer/

In [1]:
import pandas as pd

In [3]:
titanic = pd.read_csv("https://raw.githubusercontent.com/amueller/scipy-2017-sklearn/master/notebooks/datasets/titanic3.csv")
# there is still a small problem with using the OneHotEncoder and missing values,
# so for now I am going to assume there are no missing values by dropping them
titanic2 = titanic.dropna(subset=['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked'])

In [4]:
target = titanic2.survived.values
features = titanic2[['pclass', 'sex', 'age', 'fare', 'embarked']]

In [6]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043 entries, 0 to 1308
Data columns (total 5 columns):
pclass      1043 non-null int64
sex         1043 non-null object
age         1043 non-null float64
fare        1043 non-null float64
embarked    1043 non-null object
dtypes: float64(2), int64(1), object(2)
memory usage: 48.9+ KB


In [7]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer

In [8]:
preprocess = make_column_transformer(
    (['age', 'fare'], StandardScaler()),
    (['pclass', 'sex', 'embarked'], OneHotEncoder())
)

In [10]:
preprocess.fit_transform(features)

TypeError: All estimators should implement fit and transform, or can be 'drop' or 'passthrough' specifiers. '['age', 'fare']' (type <class 'list'>) doesn't.

# 2nd Example

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

In [12]:
target = titanic.survived.values
features = titanic[['pclass', 'sex', 'age', 'fare', 'embarked']].copy()
# I still fill the missing values for the embarked column, because we cannot (yet) easily handle categorical missing values
features['embarked'].fillna(features['embarked'].value_counts().index[0], inplace=True)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=0)

In [14]:
numerical_features = features.dtypes == 'float'
categorical_features = ~numerical_features

In [15]:
preprocess = make_column_transformer(
    (numerical_features, make_pipeline(SimpleImputer(), StandardScaler())),
    (categorical_features, OneHotEncoder()))

In [17]:
model = make_pipeline(
    preprocess,
    LogisticRegression())

In [18]:
model.fit(X_train, y_train)
print("logistic regression score: %f" % model.score(X_test, y_test))

C:\Users\1810837475\.conda\envs\Kompensationsarbeit\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().